## Anna Jazayeri 131661209
### Workshop 5
### Text Mining BDM 550

You may complete this workshop with a partner.<br>
In this workshop, you are to create a classifier (naïve bayes) to classify the given dataset into 6 categories: AIMX, OWNX, CONT, BASE, NUMBER, and MISC.<br><br>
Dataset:<br>
Download the corpora: labeled_dataset.zip and unlabled_dataset.zip.<br>
This corpus contains sentences from the abstract and introduction of scientific articles that have
been annotated (labeled) using the following tags:<br><br>
AIMX: "A specific research goal of the current paper"<br>
OWNX: "(Neutral) description of own work presented in current paper"<br>
CONT: "Statements of comparison with or contrast to other work; weaknesses of other work"<br>
BASE: "Statements of agreement with other work or continuation of other work"<br>
NUMBER: "Contains a word related to numbers"<br>
MISC: "(Neutral) description of other researchers' work"<br><br>
For example, the following three sentences from the labeled dataset are tagged with MISC,
AIMX, and OWNX.<br>
MISC The latter may also be applied to correct for an incidental large loop<br>
AIMX In this paper we apply the idea to graphical models for continuous variables<br>
OWNX We derive the loop corrected belief propagation equations for simple tractable<br>
Gaussian models, yielding a message passing scheme that, besides the correct average
marginals, also yields the correct variances<br><br><br>
In addition to the labeled data, this corpus contains a set of unlabeled sentences. For example, the
following two sentences are from the unlabeled dataset.<br>
here are also more than 1,100 additional genome sequencing projects currently underway
around the world CITATION, CITATION.<br>
Convenient and effective computational methods are required to handle and analyze the
immense amount of data generated by the whole-genome sequencing projects.<br><br>
Your task<br>
• Preprocess the text to remove any stop words or punctuations.<br>
• Use TF-IDF to vectorize the sentences.<br>
• Use Scikit learn to create a classifier. Then, use the classifier to label the sentences in the
unlabeled dataset.<br>
• Summarize your work and your findings in a few sentences.<br>
• BONUS [Extra 5% added to any workshops]: Use other classification approaches to label
the unlabeled sentences. Evaluate your work using precision, recall, and f1 score.<br><br>
Submit<br>
Submit your work (your code and the newly labeled dataset) on Blackboard.

In [ ]:
import nltk
import zipfile
import os
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer



In [1]:
# Import Libraries

import numpy as np
import pandas as pd
import os
import re

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import *
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import PlaintextCorpusReader

In [2]:
import nltk
import zipfile
import os
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# Download and unzip the labeled and unlabeled datasets
with zipfile.ZipFile('labeled_dataset.zip', 'r') as labeled_zip:
    labeled_zip.extractall('labeled_dataset')

with zipfile.ZipFile('unlabeled_dataset.zip', 'r') as unlabeled_zip:
    unlabeled_zip.extractall('unlabeled_dataset')

# Path to the labeled and unlabeled datasets
labeled_data_path = 'labeled_dataset'
unlabeled_data_path = 'unlabeled_dataset'

# Preprocessing function to remove stopwords and punctuations
def preprocess_text(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    # Remove stopwords and punctuations
    tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stopwords.words('english')]
    # Join the cleaned tokens into a string
    cleaned_text = ' '.join(tokens)
    return cleaned_text

# Create a list to store preprocessed labeled and unlabeled sentences
labeled_sentences = []
unlabeled_sentences = []

# Process labeled dataset
for filename in os.listdir(labeled_data_path):
    if os.path.isfile(os.path.join(labeled_data_path, filename)):
        with open(os.path.join(labeled_data_path, filename), 'r', encoding='utf-8') as file:
            for line in file:
                labeled_sentences.append(preprocess_text(line.strip()))

# Process unlabeled dataset
for filename in os.listdir(unlabeled_data_path):
    if os.path.isfile(os.path.join(unlabeled_data_path, filename)):
        with open(os.path.join(unlabeled_data_path, filename), 'r', encoding='utf-8') as file:
            for line in file:
                unlabeled_sentences.append(preprocess_text(line.strip()))

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the labeled sentences
labeled_tfidf = tfidf_vectorizer.fit_transform(labeled_sentences)

# Transform the unlabeled sentences using the same vectorizer
unlabeled_tfidf = tfidf_vectorizer.transform(unlabeled_sentences)

# Now you have TF-IDF vectors for both labeled and unlabeled sentences
print("TF-IDF vectors for labeled sentences:")
print(labeled_tfidf)

print("TF-IDF vectors for unlabeled sentences:")
print(unlabeled_tfidf)


TF-IDF vectors for labeled sentences:
  (0, 5)	1.0
  (1, 755)	0.2386719935164609
  (1, 206)	0.33470012727386117
  (1, 407)	0.2847226640982237
  (1, 426)	0.32094783589825915
  (1, 836)	0.30971139568604245
  (1, 539)	0.2847226640982237
  (1, 734)	0.33470012727386117
  (1, 280)	0.33470012727386117
  (1, 777)	0.2217585069108034
  (1, 419)	0.2620598611961059
  (1, 386)	0.18417877959019113
  (1, 35)	0.2847226640982237
  (1, 471)	0.11427806653753046
  (2, 245)	0.18161065418452957
  (2, 692)	0.2024745415731555
  (2, 629)	0.2024745415731555
  (2, 816)	0.2024745415731555
  (2, 834)	0.2024745415731555
  (2, 486)	0.2024745415731555
  (2, 338)	0.2024745415731555
  (2, 583)	0.15853131187186775
  (2, 760)	0.2024745415731555
  (2, 335)	0.15853131187186775
  (2, 541)	0.1683129130823059
  :	:
  (333, 130)	0.407459802542017
  (333, 548)	0.24910510933814353
  (333, 513)	0.24910510933814353
  (333, 186)	0.20080691515376317
  (333, 772)	0.2136587165873279
  (333, 156)	0.25698811844964015
  (333, 450)	0.1800